In [8]:
pip install --upgrade "s3fs" "aiobotocore" "fsspec" "boto3"


  Using cached boto3-1.40.63-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
  Using cached boto3-1.40.62-py3-none-any.whl.metadata (6.6 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   ------------ --------------------------- 4.5/14.1 MB 22.2 MB/s eta 0:00:01
   ---------------------------- ----------- 10.0/14.1 MB 24.9 MB/s eta 0:00:01
   ---------------------------------------- 14.1/14.1 MB 23.5 MB/s eta 0:00:00
Using cached s3transfer-0.14.0-py3-none-any.whl (85 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import s3fs

In [13]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem

In [14]:
# ---------------------------
# S3FS Setup
# ---------------------------

In [15]:
fs = s3fs.S3FileSystem(anon=False, asynchronous=False)


In [16]:
print(fs.ls('s3://stock-market-bucket-hasy'))

['stock-market-bucket-hasy/stock_market_test.json', 'stock-market-bucket-hasy/test.txt']


In [ ]:
#consumer setup

In [17]:
consumer = KafkaConsumer(
    'demotopic',
     bootstrap_servers=['3.137.139.188:9092'], #add your IP here
   # value_deserializer=lambda x: loads(x.decode('utf-8')))
    auto_offset_reset='earliest',
    enable_auto_commit=True
)

In [ ]:
#loading files to s3 bucket

In [ ]:
fs = s3fs.S3FileSystem()

for count, msg in enumerate(consumer):
    try:
        # Decode Kafka message value
        data = json.loads(msg.value.decode('utf-8'))
        
        # Create an S3 key for each file
        s3_key = f"s3://stock-market-bucket-hasy/stock_market_{count}.json"
        
        # Open the file in write mode and dump JSON content
        with fs.open(s3_key, "w") as f:
            f.write(json.dumps(data))  # ✅ dumps returns a string
        
        print(f"[{count}] Uploaded to S3 successfully: {s3_key}")
    
    except json.JSONDecodeError:
        print(f"[{count}] Skipped invalid JSON")
    except Exception as e:
        print(f"[{count}] Error: {e}")